## Практическое занятие 3. Наивный байесовский классификатор
<br><br><br><br>
__Аксентьев Артем (akseart@ya.ru)__

__Ксемидов Борис (nstalker.anonim@yandex.ru)__
<br>

In [1]:
from collections import defaultdict
import re

from numpy import log
from pandas import read_csv
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pymorphy2
from nltk.corpus import stopwords

Предобработка текста:
1. Приведение к нижнему регистру
2. Токенизация
3. Удаление стоп-слов
4. Удаление пунктуации
5. Фильтрация слов по частоте/длине/регулярному выражению
6. Лемматизация или стемминг

In [2]:
morph = pymorphy2.MorphAnalyzer()
morph.parse("бегал")

[Parse(word='бегал', tag=OpencorporaTag('VERB,impf,intr masc,sing,past,indc'), normal_form='бегать', score=1.0, methods_stack=((DictionaryAnalyzer(), 'бегал', 15, 7),))]

In [3]:
morph.parse("бегал")[0].normal_form

'бегать'

In [4]:
morph.parse("бегу")

[Parse(word='бегу', tag=OpencorporaTag('NOUN,inan,masc,Sgtm sing,loc2'), normal_form='бег', score=0.5, methods_stack=((DictionaryAnalyzer(), 'бегу', 184, 6),)),
 Parse(word='бегу', tag=OpencorporaTag('NOUN,inan,masc,Sgtm sing,datv'), normal_form='бег', score=0.25, methods_stack=((DictionaryAnalyzer(), 'бегу', 184, 2),)),
 Parse(word='бегу', tag=OpencorporaTag('VERB,perf,intr sing,1per,futr,indc'), normal_form='бежать', score=0.125, methods_stack=((DictionaryAnalyzer(), 'бегу', 392, 5),)),
 Parse(word='бегу', tag=OpencorporaTag('VERB,impf,intr sing,1per,pres,indc'), normal_form='бежать', score=0.125, methods_stack=((DictionaryAnalyzer(), 'бегу', 392, 43),))]

In [5]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("russian")
stemmer.stem("бегал")

'бега'

In [6]:
stemmer.stem("бегу")

'бег'

Алгоритм предобработки:
1. Приводим всё к нижнему регистру.
2. Удаляем из текста всё, кроме букв латиницы и кириллицы, цифр и пробелов.
3. Токенизируем по пробелам.
4. Фильтруем получившийся список токенов по стоп-словам.
5. Лемматизируем токены.

In [7]:
def clean_text(text: str, morph: pymorphy2.MorphAnalyzer,
               stop_words: list = stopwords.words('english')):

    text = text.lower()
    text = re.sub(r'[^a-z0-9а-яё\s]', ' ', text).split(' ')
    text = list(filter(None, text))
    text = [token for token in text if token not in stop_words]

    text_normalize = []
    for word in text:
        text_normalize.append(morph.parse(word)[0].normal_form)

    return text_normalize


In [8]:
morph = pymorphy2.MorphAnalyzer(lang="ru")

clean_text(
    ("Нет ничего на свете, из чего нельзя было бы сделать вывод."
     "Надо только знать, как взяться за дело."), morph, stop_words=stopwords.words("russian"))

['свет', 'сделать', 'вывод', 'знать', 'взяться', 'дело']

In [9]:
def clean_texts(texts):
    morph = pymorphy2.MorphAnalyzer()
    return [clean_text(text, morph) for text in texts]

In [10]:
cleaned_texts = clean_texts(
    [
        "Алиса, надо сказать, частенько давала себе очень разумные советы, но довольно редко следовала им.",
        "Впрочем, ведь раз некому ответить, то не всё ли равно, о чём спрашивать?"
    ]
)
print(cleaned_texts)

[['алиса', 'надо', 'сказать', 'частенько', 'давать', 'себя', 'очень', 'разумный', 'совет', 'но', 'довольно', 'редко', 'следовать', 'они'], ['впрочем', 'ведь', 'раз', 'некого', 'ответить', 'то', 'не', 'всё', 'ли', 'равно', 'о', 'что', 'спрашивать']]


Условная вероятность:

$P(AB) = P(A|B) P(B) = P(B|A) P(A)$

$P(A|B) = \frac{P(AB)}{P(B)} = \frac{P(B|A) P(A)}{P(B)}$

Теорема Байеса (вероятность события $А$ при наступлении события $B$ или апостериорная вероятность):

$P(A|B) = \frac{P(A) P(B|A)}{P(B)}$

Пусть нам дан вектор признаков $x = (x_1, x_2, x_3, ..., x_n)$ и некоторый набор классов $C_1, C_2, ..., C_k$. Тогда используя теорему Байеса можно записать:

$P(C_k | x) = \frac{P(C_k) P(x | C_k)}{P(x)}$

Таким образом, мы можем получить вероятности отнесения конкретного объекта к каждому классу и выбрать наиболее вероятный.

$P(C_k | x_1, x_2, x_3, ..., x_n) = P(C_k) P(x_1 | C_k) P(x_2 | C_k)\ ...\ P(x_n | C_k)$

Тогда задача сводится к

$y = \arg \max\limits_{C_k} P(C_k) P(x_1 | C_k) P(x_2 | C_k)\ ...\ P(x_n | C_k) = \arg \max\limits_{C_k} P(C_k)  ∏\limits_n p(x_i | C_k)$ 

Для облегчения математических операций прологарифмируем задачу (и для большей "числовой" стабильности):

$y = \arg \max\limits_{C_k} P(C_k)  ∏\limits_n p(x_i | C_k) = \arg \max\limits_{C_k} \log P(C_k) \sum\limits_n \log(p(x_i|C_k))$

**Примечание:** сделать нам это позволяет монотонность логарифма (всё ещё ищем максимум) и свойство логарифма: $\log (a b) = \log a + \log b$

В полиномиальном наивном баейсовском классификаторе оценки вероятностей $P(x_i|C_k)$ считаются иначе:

$P(x_i|C_k) = \frac{N_{C_k i} + \alpha}{N_{C_k} + \alpha n}$, где
- $\alpha$ - коэффициент сглаживания;
- $N_{C_k}$ - количество встреченных объектов класса $C_k$;
- $N_{C_k i}$ - количество вхождений i-го слова в объектах класса $C_k$;
- $n$ - размер словаря или кол-во признаков.

**Алгоритм обучения**:
1. Считаем количество объектов каждого класса ($N_{C_k}$), количества каждого слова для каждого класса ($N_{C_k i}$) и составляем словарь слов.
2. Считаем логарифмы оценок вероятностей каждого класса ($\log(P(C_k)) = \log(\frac{N_{C_k}}{n})$, где $n$ - размер датасета, а $N_{C_k}$ - количество встреченных объектов класса $C_k$).

**Алгоритм предсказания**:
- Для каждого объекта выполняем:
    - для каждого класса, перебирая слова ($x_i$), вычисляем $\log P(C_k) \sum\limits_n \log(p(x_i|C_k))$ и выбираем класс с максимальный значением;
    - сохраняем предсказание для текущего объекта.

In [11]:
from collections import Counter

import numpy as np


class Bayes(BaseEstimator, ClassifierMixin):
    def __init__(self, alpha):
        self.alpha = alpha
        self.vocab = set()
        # Счётчик классов в данных
        self.classes = defaultdict(lambda: 0)
        # Счётчик для каждого слова при каждом классе
        self.feats_counts = defaultdict(lambda: 0)
        # Прологарифмированные оценки вероятностей классов
        self.log_classes_freq = defaultdict(lambda: 0)

    def fit(self, x, y):
        for feats, label in zip(x, y):
            self.classes[label] += 1
            for feat, count in Counter(feats).items():
                if feat not in self.vocab:
                    self.vocab.add(feat)
                self.feats_counts[label, feat] += count

        n = len(x)
        for c in self.classes:
            self.log_classes_freq[c] = np.log(self.classes[c] / n)

    def predict(self, x):
        preds = []
        for feats in x:
            classes_scores = {c: 0 for c in self.classes.keys()}
            for word in Counter(feats).keys():
                if word not in self.vocab:
                    continue

                for c in self.classes.keys():
                    classes_scores[c] += np.log((self.feats_counts[c, word] + self.alpha) / (
                        self.classes[c] + self.alpha * len(self.vocab)))

            for c in self.classes.keys():
                classes_scores[c] += self.log_classes_freq[c]

            preds.append(max(classes_scores.items(), key=lambda x: x[1])[0])

        return preds
        

In [12]:

transformer = FunctionTransformer(clean_texts)
pipeline = Pipeline([('t', transformer),
                     ('cls', Bayes(1))])

df = read_csv("./train.csv")

x = df['text'].tolist()
y = df['target'].tolist()

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=42)

pipeline.fit(x_train, y_train)
pred = pipeline.predict(x_test)

print('Accuracy =', accuracy_score(pred, y_test))


Accuracy = 0.7951680672268907


In [13]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

morph = pymorphy2.MorphAnalyzer()
transformer = FunctionTransformer(
    lambda data: [" ".join(clean_text(x, morph=morph)) for x in data])

pipeline = Pipeline([('t', transformer),
                     ("vectorizer", CountVectorizer()),
                     ('cls', MultinomialNB(alpha=1))])

pipeline.fit(x_train, y_train)
pred = pipeline.predict(x_test)

print('Accuracy =', accuracy_score(pred, y_test))

Accuracy = 0.7909663865546218


Домашнее задание:
1. Скачать датасет (https://www.kaggle.com/datasets/yasserh/imdb-movie-ratings-sentiment-analysis).
2. Предобработать текст в колонке text (подумать, как именно это можно сделать).
3. Разделить выборку на обучающую и тестовую, используя стратификацию.
4. Создать пайплайн для обучения с использованием полиномиального наивного байесовского классификатора из библиотеки sklearn и различных подходов к векторизации текста (CountVectorizer, TfidfVectorizer) и т.д + сравнить их.
5. Обучить модель и оценить кач-во с помощью метрик accuracy и f1-score.